In [1]:
import requests
import json
import random
import time
import pickle


model = "dRIT"

template = {
  "Answer": "",
  "Score Lexical Semantics": "",
  "Score Predicate-Argument Structure": "",
  "Score Logic": "",
  "Score Knowledge & Common Sense": ""
}


# TRAIN de MultiNLI

In [2]:
import pandas as pd

In [3]:
df =pd.read_pickle("predictions/RL_MultiNLI_DEVMM.pickle")
df

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases
0,-1,0.400000,0.2,0.750000,0.721688,0.231544,4.714186,1.000000,1.000000,0,0,0.819444,0,0,0.400000,contradiction
1,1,0.000000,0.0,1.000000,0.289570,0.064122,4.525959,0.942195,1.000000,1,0,0.652381,1,0,1.000000,contradiction
2,1,0.000000,0.0,1.000000,0.645497,0.228033,4.607561,1.000000,1.000000,0,0,0.711111,0,0,1.000000,entailment
3,2,0.500000,0.0,0.500000,0.306186,0.013498,4.185469,0.559530,0.428571,1,0,0.486111,0,1,0.500000,contradiction
4,-1,0.400000,0.2,0.833333,0.816497,0.300323,4.675483,1.000000,1.000000,0,0,0.644444,1,1,0.400000,entailment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2,1.000000,0.0,0.500000,0.707107,0.529298,3.833234,0.529298,1.000000,1,0,0.000000,0,0,0.000000,contradiction
9996,2,0.333333,0.0,0.666667,0.548580,0.051191,4.688329,0.946973,0.846154,0,1,0.732143,0,0,0.666667,contradiction
9997,2,0.333333,0.0,0.400000,0.400000,0.154047,4.418746,0.775202,0.750000,0,0,0.604762,0,1,0.666667,contradiction
9998,2,1.000000,0.0,0.000000,0.258199,0.100660,3.879200,0.396843,0.500000,1,0,0.511111,0,1,0.000000,neutral


In [4]:
original =pd.read_csv("../OPENAI/data/MultiNLI/DEVMM/DEV_1.csv")
original

,sentence1,sentence2,gold_label
0,Your contribution helped make it possible for ...,Your contributions were of no help with our st...,contradiction
1,"The answer has nothing to do with their cause,...",Dictionaries are indeed exercises in bi-unique...,contradiction
2,We serve a classic Tuscan meal that includes ...,We serve a meal of Florentine terrine.,entailment
3,"A few months ago, Carl Newton and I wrote a le...",Carl Newton and I have never had any other pre...,contradiction
4,"I was on this earth you know, I've lived on th...",I don't yet know the reason why I have lived o...,entailment
...,...,...,...
9995,Do you watch that?,Can you see?,contradiction
9996,"To a Western ear, the most predictable of lang...","To the Western ear, the least predictable of l...",contradiction
9997,The recorder captured the sounds of loud thump...,The recorder didn't capture any of the sounds.,contradiction
9998,That's a good attitude!,"You feel good about this, don't you?",neutral


In [5]:
df["sentence_A"]=original["sentence1"]
df["sentence_B"]=original["sentence2"]

In [6]:
entailment=df[df["clases"]=="entailment"]
neutral=df[df["clases"]=="neutral"]
contradiction=df[df["clases"]=="contradiction"]
contradiction

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
0,-1,0.400000,0.20,0.750000,0.721688,0.231544,4.714186,1.000000,1.000000,0,0,0.819444,0,0,0.400000,contradiction,Your contribution helped make it possible for ...,Your contributions were of no help with our st...
1,1,0.000000,0.00,1.000000,0.289570,0.064122,4.525959,0.942195,1.000000,1,0,0.652381,1,0,1.000000,contradiction,"The answer has nothing to do with their cause,...",Dictionaries are indeed exercises in bi-unique...
3,2,0.500000,0.00,0.500000,0.306186,0.013498,4.185469,0.559530,0.428571,1,0,0.486111,0,1,0.500000,contradiction,"A few months ago, Carl Newton and I wrote a le...",Carl Newton and I have never had any other pre...
6,2,1.000000,0.00,0.500000,0.353553,-0.013505,3.780560,0.123214,0.333333,1,1,0.000000,0,0,0.000000,contradiction,"I'll twist him, sir.",I'll make him straight.
8,-1,0.250000,0.25,0.666667,0.493197,0.106861,4.707954,1.000000,1.000000,0,0,0.553333,0,0,0.500000,contradiction,"For more than a century, the U.S. federal and ...",There are no sweatshops in the garment industry.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9990,2,0.666667,0.00,0.500000,0.408248,-0.080891,2.509210,0.286103,0.666667,1,0,0.527778,0,0,0.333333,contradiction,"For Indianapolis, that public university must ...",IUPUI is in the city of Chicago.
9994,2,0.857143,0.00,0.000000,0.258199,0.080466,3.890266,0.320655,0.125000,1,1,0.000000,0,0,0.142857,contradiction,Please donate today.,Our website is down please come back tomorrow ...
9995,2,1.000000,0.00,0.500000,0.707107,0.529298,3.833234,0.529298,1.000000,1,0,0.000000,0,0,0.000000,contradiction,Do you watch that?,Can you see?
9996,2,0.333333,0.00,0.666667,0.548580,0.051191,4.688329,0.946973,0.846154,0,1,0.732143,0,0,0.666667,contradiction,"To a Western ear, the most predictable of lang...","To the Western ear, the least predictable of l..."


In [7]:
df_e=entailment.sample(n=200)
df_n=neutral.sample(n=200)
df_c=contradiction.sample(n=200)
df_t=pd.concat([df_e,df_n,df_c])
df_t

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
4501,2,0.250000,0.0,0.500000,0.500000,0.172598,3.440214,0.750000,1.000000,0,0,0.464286,0,0,0.750000,entailment,"If you help me, I said, to move my livestock t...",Can you help me move my livestock to Chapeltique?
5804,2,0.250000,0.0,0.750000,0.564076,0.172460,4.467687,0.875487,1.000000,2,0,0.674272,1,1,0.750000,entailment,Make-believe roles are not equally pleasurable...,Not all make-believe role-playing is equally a...
3970,2,0.833333,0.0,0.400000,0.316228,0.009732,3.962749,0.371995,0.500000,2,0,0.458333,0,0,0.166667,entailment,He had an appetite and aptitude for business.,"In business, he found that he could excel; on ..."
5871,2,0.250000,0.0,0.333333,0.368721,0.073688,4.783562,1.000000,1.000000,0,0,0.308547,1,0,0.750000,entailment,I see the rather boring point about using alum...,The term prima donna can be used for men.
4268,2,0.500000,0.0,0.500000,0.912871,0.258084,4.631388,0.838558,1.000000,1,0,0.944444,0,0,0.500000,entailment,How was it going to high school in Charlotte?,What was going to high school in Charlotte like?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,2,0.222222,0.0,0.500000,0.734847,0.105542,4.605025,0.903593,0.888889,0,0,0.827066,0,0,0.777778,contradiction,The line to the White House shelter conference...,The line to the White House shelter conference...
594,2,0.250000,0.0,0.250000,0.544331,0.026523,4.411366,0.862340,0.833333,1,0,0.703704,0,0,0.750000,contradiction,"Making a marker is a complicated task, even wi...",Making markers is easy even without the help o...
447,2,0.750000,0.0,0.000000,0.327327,0.042057,4.256955,0.560480,0.600000,1,0,0.492063,0,1,0.250000,contradiction,We can and should be proud that the IU School ...,The UI school of dentistry doesn't help anyone.
255,2,1.000000,0.0,0.500000,0.547723,0.082198,3.422672,0.478159,0.400000,0,0,0.588889,0,0,0.000000,contradiction,Representatives from 43 different countries we...,Only 10 different representatives showed up.


In [8]:
lista_respuestasOllama=[]

In [9]:
df.columns

Index(['relation', 'no_matcheadas', 'contradiction', 'jaccard', 'simBoW',
       'sumas', 'mutinf_t', 'max_info_t', 'entail', 'list_comp', 'list_incomp',
       'Jaro-Winkler_rit', 'negT', 'negH', 'overlap_ent', 'clases',
       'sentence_A', 'sentence_B'],
      dtype='object')

In [10]:
faltantes=[]

In [11]:
df_t=df_t.reset_index(drop=True)
df_t

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
0,2,0.250000,0.0,0.500000,0.500000,0.172598,3.440214,0.750000,1.000000,0,0,0.464286,0,0,0.750000,entailment,"If you help me, I said, to move my livestock t...",Can you help me move my livestock to Chapeltique?
1,2,0.250000,0.0,0.750000,0.564076,0.172460,4.467687,0.875487,1.000000,2,0,0.674272,1,1,0.750000,entailment,Make-believe roles are not equally pleasurable...,Not all make-believe role-playing is equally a...
2,2,0.833333,0.0,0.400000,0.316228,0.009732,3.962749,0.371995,0.500000,2,0,0.458333,0,0,0.166667,entailment,He had an appetite and aptitude for business.,"In business, he found that he could excel; on ..."
3,2,0.250000,0.0,0.333333,0.368721,0.073688,4.783562,1.000000,1.000000,0,0,0.308547,1,0,0.750000,entailment,I see the rather boring point about using alum...,The term prima donna can be used for men.
4,2,0.500000,0.0,0.500000,0.912871,0.258084,4.631388,0.838558,1.000000,1,0,0.944444,0,0,0.500000,entailment,How was it going to high school in Charlotte?,What was going to high school in Charlotte like?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,2,0.222222,0.0,0.500000,0.734847,0.105542,4.605025,0.903593,0.888889,0,0,0.827066,0,0,0.777778,contradiction,The line to the White House shelter conference...,The line to the White House shelter conference...
596,2,0.250000,0.0,0.250000,0.544331,0.026523,4.411366,0.862340,0.833333,1,0,0.703704,0,0,0.750000,contradiction,"Making a marker is a complicated task, even wi...",Making markers is easy even without the help o...
597,2,0.750000,0.0,0.000000,0.327327,0.042057,4.256955,0.560480,0.600000,1,0,0.492063,0,1,0.250000,contradiction,We can and should be proud that the IU School ...,The UI school of dentistry doesn't help anyone.
598,2,1.000000,0.0,0.500000,0.547723,0.082198,3.422672,0.478159,0.400000,0,0,0.588889,0,0,0.000000,contradiction,Representatives from 43 different countries we...,Only 10 different representatives showed up.


In [12]:
df_t.to_pickle("predictions/RL_MultiNLI_DEVMM_600.pickle")

# RIT base

In [13]:
for i in range(1):
    for index,strings in df_t[i*600:(i+1)*600].iterrows():
        prompt = ''''Analyze the following sentences: 
        Text: '''+strings["sentence_A"]+'''
        Hypothesis: '''+ strings["sentence_B"]+'''
        only responds using the template. '''
        #print(prompt)
        data = {
            "prompt": prompt,
            "model": model,
            "format": "json",
            "stream": False,
            #"options": {"temperature": 0.2},
        }
        try:
            response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=15)        
            json_data = json.loads(response.text)
            lista_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
            print(index)
        except:
            print("Saltó",index)
            faltantes.append(index)
            lista_respuestasOllama.append("NA")
    with open("resultados/rit_base_MultiNLI_DEVMM_600_"+str(i+1)+".pickle", "wb") as f:
        pickle.dump(lista_respuestasOllama, f)
    print("Primer rango: ",(i)*600,(i+1)*600)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [14]:
print(len(lista_respuestasOllama))

600
